# Cassava Leaf Disease Modelling
There is already a great kernel [Vision Transformer (ViT): Tutorial + Baseline](https://www.kaggle.com/abhinand05/vision-transformer-vit-tutorial-baseline) that shows us how to use visiontransformer on TPU and why. Here, we make further improvements on the basis of the official implementation, in order to provide better pre-training parameters and user-friendly API as `Effecientnet-PyTorch`. You can find all the details on https://github.com/tczhangzhi/VisionTransformer-Pytorch.


In [ ]:
import sys

package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
for pth in package_paths:
    sys.path.append(pth)
import timm


In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

In [ ]:
# Import libraries
import os
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
import json
import seaborn as sns
import cv2
import albumentations as albu
import numpy as np

In [ ]:
device = torch.device('cuda:0')

# Visualization

# Modelling

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split
from albumentations.pytorch import ToTensorV2
# from efficientnet_pytorch import EfficientNet
import time
import datetime
import copy

In [ ]:
CFG = {
    'fold_num': 10,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b3_ns',
    'img_size': 384,
    'epochs': 32,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 4,
    'used_epochs': [8],#[6,7,8,9],
    'weights': [1,1,1,1,1]
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [ ]:
# DataSet class

class CassavaDataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms
        
    def __getitem__(self,index):
        im_path=os.path.join(self.imfolder,self.df.iloc[index]['image_id'])
        x=cv2.imread(im_path,cv2.IMREAD_COLOR)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        
        if(self.transforms):
            x=self.transforms(image=x)['image']
        
        if(self.train):
            y=self.df.iloc[index]['label']
            return x,y
        else:
            return x
        
    def __len__(self):
        return len(self.df)

In [ ]:
#from vision_transformer_pytorch import VisionTransformer

# model_name = 'efficientnet-b7'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CustomDeiT(nn.Module):
    def __init__(self, model_name='model_name', pretrained=False):
        super().__init__()
        self.model = torch.hub.load('facebookresearch/deit:main', model_name, pretrained=0)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        print(self.model)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x
    
class CustomViT(nn.Module):
    def __init__(self, model_name='', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features,5)

    def forward(self, x):
        x = self.model(x)
        return x
# model = create_model(
#         'deit_base_patch16_224',
#         pretrained=False,
#         num_classes=5,
        
#         drop_block_rate=None,
#     ).to(device)
# model=models.(pretrained=True)
# model.fc=nn.Linear(512,5)
# model = EfficientNet.from_pretrained(model_name, num_classes=5) 
# model=models.resnext50_32x4d()#Add Pretrained=True to use pretrained with internet enabled
# model.fc=nn.Linear(model.fc.in_features,5)
model = CustomViT(model_name='vit_base_patch16_384', pretrained=False).to(device)
device = torch.device(CFG['device'])
model.to(device)
model.eval()



Train the model after uncommenting below

In [ ]:
# trained_model=train_model(datasets,dataloaders,model,criterion,optimizer,scheduler,num_epochs,device)

In [ ]:
# Epoch 0/5
# ----------
# train Loss: 0.5318 Acc: 0.8119
# valid Loss: 0.4009 Acc: 0.8650

# Epoch 1/5
# ----------
# train Loss: 0.4384 Acc: 0.8467
# valid Loss: 0.3999 Acc: 0.8612

# Epoch 2/5
# ----------
# train Loss: 0.3511 Acc: 0.8778
# valid Loss: 0.3558 Acc: 0.8771

# Epoch 3/5
# ----------
# train Loss: 0.3266 Acc: 0.8879
# valid Loss: 0.3468 Acc: 0.8836

# Epoch 4/5
# ----------
# train Loss: 0.3066 Acc: 0.8924
# valid Loss: 0.3384 Acc: 0.8911

# Epoch 5/5
# ----------
# train Loss: 0.3060 Acc: 0.8926
# valid Loss: 0.3421 Acc: 0.8869

# Training complete in 121m 52s
# Best val Acc: 0.891121

Save the model after training

Load the model when model is trained and saved and notebook has to be run without internet

# Submission

In [ ]:

class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [ ]:
from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,RandomCrop, ShiftScaleRotate, CenterCrop, Resize
)

def get_inference_transforms(CFG):
    return Compose([
            CenterCrop(512,512),
            Resize(CFG['img_size'], CFG['img_size']),
           Transpose(p=0.5),
           HorizontalFlip(p=0.5),
           VerticalFlip(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
import glob
from tqdm import tqdm
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all
if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])
    
#     folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values)
    tst_preds = []
        
    if 1:
        # we'll train fold 0 first


        test = pd.DataFrame()
        test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms(CFG), output_label=False)

        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )

        device = torch.device(CFG['device'])
#         if CFG['model_arch'] == 'resnext50_32x4d':
#             model = CustomResNext().to(device)
#         if 'efficientnet' in CFG['model_arch']:
#             model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique()).to(device)

        #for epoch in range(CFG['epochs']-3):
        for (i,path) in enumerate(glob.glob('../input/vit5fold/*.pth')):    
            model.load_state_dict(torch.load(path)['model'])
            
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    #val_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader, device)]
                    tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

        #val_preds = np.mean(val_preds, axis=0)         
        #print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_.label.values, val_preds)))
       #print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_.label.values==np.argmax(val_preds, axis=1)).mean()))
        del model
        torch.cuda.empty_cache()

In [ ]:
tst_preds = np.mean(tst_preds, axis=0) 


In [ ]:
test['label'] = np.argmax(tst_preds, axis=1)
test.head()

In [ ]:
test.to_csv('submission.csv', index=False)